# Basic Introduction to Theano


In [ ]:
Tomasz Golan
MLVF Meeting, 03.16.2016

# Linear Regression

# Notation

* Features: $$x = (x_1, ..., x_n)$$
* Training example: $$(x^{(i)}, y^{(i)})$$
* Training set: $$\left\{(x^{(i)}, y^{(i)}); i = 1, ..., N\right\}$$

# Notation

* Weights: $$w = (w_0, w_1, ..., w_n)$$
* For convenience: $$x_0 = 1$$
* Hypothesis: $$h (x) = w_0 + w_1x_1 + ... + w_nx_n = \sum\limits_{i = 0}^n w_i x_i = w^T x$$

In [2]:
from IPython.display import display, HTML
s = """

<style>

.rendered_html h1 {
    color: #015C9C;
    text-align: center;
}

</style>
"""
display(HTML(s))